In [1]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)

source("src/paths/paths.R")
path_er <- file.path(path.ds, "ARPA", "EMILIA-ROMAGNA")


In [2]:
ds <- read_json_arrow(file.path(path_er, "2016-01.json.gz"), as_data_frame = FALSE)


In [3]:
ds


Table
1121649 rows x 7 columns
$version <string>
$network <string>
$ident <null>
$lon <int64>
$lat <int64>
$date <timestamp[s]>
$data: list<item: struct<vars: struct<B01019: struct<v: string>, B01194: struct<v: string>, B04001: struct<v: int64>, B04002: struct<v: int64>, B04003: struct<v: int64>, B04004: struct<v: int64>, B04005: struct<v: int64>, B04006: struct<v: int64>, B05001: struct<v: double>, B06001: struct<v: double>, B07030: struct<v: double>, B07031: struct<v: double>, B13011: struct<v: double, a: struct<B33007: int64>>, B12101: struct<v: double, a: struct<B33007: int64>>, B13003: struct<v: int64, a: struct<B33007: int64>>, B11001: struct<v: int64, a: struct<B33007: int64>>, B11002: struct<v: double, a: struct<B33007: int64>>, B11041: struct<v: double, a: struct<B33007: int64>>, B14198: struct<v: int64, a: struct<B33007: int64>>, B10004: struct<v: int64>, B13013: struct<v: double, a: struct<B33007: int64>>, B13215: struct<v: double, a: struct<B33007: int64>>, B11043: struct<v

In [53]:
library(tidyr)
vrs <- ds |>
    slice_head(n = 10) |>
    collect() |>
    unnest_wider(data) |>
    select(date, vars, timerange, level) |>
    unnest_longer(c(vars, timerange, level)) |>
    unnest_wider(vars)
# unnest_wider(c(timerange, level), names_sep = "_") |>
# pull(vars)


In [54]:
vrs |>
    select(where(~ any(!is.na(.)))) |>
    select(!c(B01194, starts_with("B0400"), starts_with("B05"), starts_with("B06"), starts_with("B07"))) |> # metadata
    select(!c(B13011, B13003)) |> # data (precip, humidity)
    unnest_wider(starts_with("B"), names_sep = "_") |>
    select(!ends_with("_a")) |> 
    filter(!if_all(starts_with("B"), is.na)) |>
    unnest_wider(c(level, timerange), names_sep = "_") |>
    select(!c(level_3, level_4))


date,B01019_v,B12101_v,timerange_1,timerange_2,timerange_3,level_1,level_2
<dttm>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>
2016-01-01 01:00:00,Carpineti,NA,NA,NA,NA,NA,NA
2016-01-01 01:00:00,NA,273.85,0,0,3600,103,2000
2016-01-01 01:00:00,NA,275.21,0,0,86400,103,2000
2016-01-01 01:00:00,NA,274.25,2,0,3600,103,2000
2016-01-01 01:00:00,NA,277.75,2,0,86400,103,2000
2016-01-01 01:00:00,NA,273.45,3,0,3600,103,2000
2016-01-01 01:00:00,NA,272.75,3,0,86400,103,2000
2016-01-01 01:00:00,NA,273.75,254,0,0,103,2000
2016-01-01 01:15:00,Carpineti,NA,NA,NA,NA,NA,NA


In [28]:
ds |>
    slice_head(n = 1) |>
    collect() |>
    unnest_wider(data) |>
    hoist(vars, TEMP = "B12101") |>
    unnest_longer(c(TEMP, timerange, level)) |>
    unnest_wider(TEMP) |>
    unnest_longer(timerange) |>
    select(date, TEMP = v, timerange, level) |>
    unnest_longer(level) |>
    drop_na(TEMP) |>
    print()


# A tibble: 84 x 4
   date                 TEMP timerange level
   <dttm>              <dbl>     <int> <int>
 1 2016-01-01 01:00:00  274.         0   103
 2 2016-01-01 01:00:00  274.         0  2000
 3 2016-01-01 01:00:00  274.         0    NA
 4 2016-01-01 01:00:00  274.         0    NA
 5 2016-01-01 01:00:00  274.         0   103
 6 2016-01-01 01:00:00  274.         0  2000
 7 2016-01-01 01:00:00  274.         0    NA
 8 2016-01-01 01:00:00  274.         0    NA
 9 2016-01-01 01:00:00  274.      3600   103
10 2016-01-01 01:00:00  274.      3600  2000
# i 74 more rows


In [3]:
sample <- read_json_arrow(file.path(path_er, "2016-01.json"), as_data_frame = FALSE) |>
    slice_head(n = 5000) |>
    as_tibble()


In [4]:
sample


version network ident lon     lat     date               
1    0.1     agrmet  NA    1050865 4447562 2016-01-01 01:00:00
2    0.1     agrmet  NA    1050865 4447562 2016-01-01 01:15:00
3    0.1     agrmet  NA    1050865 4447562 2016-01-01 01:30:00
4    0.1     agrmet  NA    1050865 4447562 2016-01-01 01:45:00
5    0.1     agrmet  NA    1050865 4447562 2016-01-01 02:00:00
6    0.1     agrmet  NA    1050865 4447562 2016-01-01 02:15:00
7    0.1     agrmet  NA    1050865 4447562 2016-01-01 02:30:00
8    0.1     agrmet  NA    1050865 4447562 2016-01-01 02:45:00
9    0.1     agrmet  NA    1050865 4447562 2016-01-01 03:00:00
10   0.1     agrmet  NA    1050865 4447562 2016-01-01 03:15:00
11   0.1     agrmet  NA    1050865 4447562 2016-01-01 03:30:00
12   0.1     agrmet  NA    1050865 4447562 2016-01-01 03:45:00
13   0.1     agrmet  NA    1050865 4447562 2016-01-01 04:00:00
14   0.1     agrmet  NA    1050865 4447562 2016-01-01 04:15:00
15   0.1     agrmet  NA    1050865 4447562 2016-01-01 04:30:00
16   0.1     agrmet  NA    1050865 4447562 2016-01-01 04:45:00
17   0.1     agrmet  NA    1050865 4447562 2016-01-01 05:00:00
18   0.1     agrmet  NA    1050865 4447562 2016-01-01 05:15:00
19   0.1     agrmet  NA    1050865 4447562 2016-01-01 05:30:00
20   0.1     agrmet  NA    1050865 4447562 2016-01-01 05:45:00
21   0.1     agrmet  NA    1050865 4447562 2016-01-01 06:00:00
22   0.1     agrmet  NA    1050865 4447562 2016-01-01 06:15:00
23   0.1     agrmet  NA    1050865 4447562 2016-01-01 06:30:00
24   0.1     agrmet  NA    1050865 4447562 2016-01-01 06:45:00
25   0.1     agrmet  NA    1050865 4447562 2016-01-01 07:00:00
26   0.1     agrmet  NA    1050865 4447562 2016-01-01 07:15:00
27   0.1     agrmet  NA    1050865 4447562 2016-01-01 07:30:00
28   0.1     agrmet  NA    1050865 4447562 2016-01-01 07:45:00
29   0.1     agrmet  NA    1050865 4447562 2016-01-01 08:00:00
30   0.1     agrmet  NA    1050865 4447562 2016-01-01 08:15:00
...  ...     ...     ...   ...     ...     ...                
4971 0.1     agrmet  NA    1159701 4441112 2016-01-21 19:30:00
4972 0.1     agrmet  NA    1159701 4441112 2016-01-21 19:45:00
4973 0.1     agrmet  NA    1159701 4441112 2016-01-21 20:00:00
4974 0.1     agrmet  NA    1159701 4441112 2016-01-21 20:15:00
4975 0.1     agrmet  NA    1159701 4441112 2016-01-21 20:30:00
4976 0.1     agrmet  NA    1159701 4441112 2016-01-21 20:45:00
4977 0.1     agrmet  NA    1159701 4441112 2016-01-21 21:00:00
4978 0.1     agrmet  NA    1159701 4441112 2016-01-21 21:15:00
4979 0.1     agrmet  NA    1159701 4441112 2016-01-21 21:30:00
4980 0.1     agrmet  NA    1159701 4441112 2016-01-21 21:45:00
4981 0.1     agrmet  NA    1159701 4441112 2016-01-21 22:00:00
4982 0.1     agrmet  NA    1159701 4441112 2016-01-21 22:15:00
4983 0.1     agrmet  NA    1159701 4441112 2016-01-21 22:30:00
4984 0.1     agrmet  NA    1159701 4441112 2016-01-21 22:45:00
4985 0.1     agrmet  NA    1159701 4441112 2016-01-21 23:00:00
4986 0.1     agrmet  NA    1159701 4441112 2016-01-21 23:15:00
4987 0.1     agrmet  NA    1159701 4441112 2016-01-21 23:30:00
4988 0.1     agrmet  NA    1159701 4441112 2016-01-21 23:45:00
4989 0.1     agrmet  NA    1159701 4441112 2016-01-22 00:00:00
4990 0.1     agrmet  NA    1159701 4441112 2016-01-22 00:15:00
4991 0.1     agrmet  NA    1159701 4441112 2016-01-22 00:30:00
4992 0.1     agrmet  NA    1159701 4441112 2016-01-22 00:45:00
4993 0.1     agrmet  NA    1159701 4441112 2016-01-22 01:00:00
4994 0.1     agrmet  NA    1159701 4441112 2016-01-22 01:15:00
4995 0.1     agrmet  NA    1159701 4441112 2016-01-22 01:30:00
4996 0.1     agrmet  NA    1159701 4441112 2016-01-22 01:45:00
4997 0.1     agrmet  NA    1159701 4441112 2016-01-22 02:00:00
4998 0.1     agrmet  NA    1159701 4441112 2016-01-22 02:15:00
4999 0.1     agrmet  NA    1159701 4441112 2016-01-22 02:30:00
5000 0.1     agrmet  NA    1159701 4441112 2016-01-22 02:45:00
     data                                                                                          